In [3]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
test_unknown_data_path = './test_unknown_data.csv'
test_unknown_data = pd.read_csv(test_unknown_data_path)

test_unknown_label_path = './test_unknown_label.csv'
test_unknown_label = pd.read_csv(test_unknown_label_path)

In [5]:
# Display column names
print("Column names:")
print(test_unknown_data.columns)

# Display dataset shape
print("\nDataset shape:")
print(test_unknown_data.shape)


Column names:
Index(['flow_duration', 'Header_Length', 'Protocol Type', 'Duration', 'Rate',
       'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count', 'syn_count',
       'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet',
       'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC',
       'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number',
       'Magnitue', 'Radius', 'Covariance', 'Variance', 'Weight'],
      dtype='object')

Dataset shape:
(5953, 46)


In [7]:
# Print data types
print("\nData Types:")
print(test_unknown_data.dtypes)


Data Types:
flow_duration      float64
Header_Length      float64
Protocol Type      float64
Duration           float64
Rate               float64
Srate              float64
Drate                int64
fin_flag_number      int64
syn_flag_number      int64
rst_flag_number      int64
psh_flag_number      int64
ack_flag_number      int64
ece_flag_number      int64
cwr_flag_number      int64
ack_count          float64
syn_count          float64
fin_count          float64
urg_count          float64
rst_count          float64
HTTP                 int64
HTTPS                int64
DNS                  int64
Telnet               int64
SMTP                 int64
SSH                  int64
IRC                  int64
TCP                  int64
UDP                  int64
DHCP                 int64
ARP                  int64
ICMP                 int64
IPv                  int64
LLC                  int64
Tot sum            float64
Min                float64
Max                float64
AVG            

In [8]:
print("Number of rows in X_train:", X_train.shape[0])
print("Number of rows in test_unknown_data:", test_unknown_data.shape[0])

NameError: name 'X_train' is not defined